# Reading JSON and CSV

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Reading CSV using API
Here the file is stores in cache and gets updated to the latest version everytime the API is called using 'kagglehub.download'

In [14]:
# import using API instead of using a locally used file
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arshid/iris-flower-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/vivektandale/.cache/kagglehub/datasets/arshid/iris-flower-dataset/versions/1


In [15]:
df = pd.read_csv(path+'/IRIS.csv')

### Importing a JSON file via API

In [16]:
# Download latest version
path2 = kagglehub.dataset_download("rtatman/iris-dataset-json-version")

print("Path to dataset files:", path2)

df2 = pd.read_json(path2+'/IRIS.json')

Path to dataset files: /Users/vivektandale/.cache/kagglehub/datasets/rtatman/iris-dataset-json-version/versions/1


### Reading Data from a SQL database using a connection

In [17]:
import sqlite3 as sq3
import pandas.io.sql as pds

This code downloads the data from server and saves it in a folder called 'data'

In [18]:
# !wget -P data https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0232EN-SkillsNetwork/asset/classic_rock.db

'con' is a an object which has live connection with the database

In [19]:
path3 = 'data/classic_rock.db'
con = sq3.Connection(path3)

In [20]:
con

In [21]:
query = '''
select * from rock_songs
limit 10
'''

observations = pds.read_sql(query, con)
observations.head()

,Song,Artist,Release_Year,PlayCount
0,Caught Up in You,.38 Special,1982.0,82
1,Hold On Loosely,.38 Special,1981.0,85
2,Rockin' Into the Night,.38 Special,1980.0,18
3,Art For Arts Sake,10cc,1975.0,1
4,Kryptonite,3 Doors Down,2000.0,13


In [22]:
query = '''
select artist, release_year, count(*) as num_of_songs, avg(playcount) as avg_play_time
from rock_songs
group by artist, release_year
order by num_of_songs desc
'''

observations = pds.read_sql(query, con)
observations

,Artist,Release_Year,num_of_songs,avg_play_time
0,The Beatles,1967.0,23,6.565217
1,Led Zeppelin,1969.0,18,21.000000
2,The Beatles,1965.0,15,3.800000
3,The Beatles,1968.0,13,13.000000
4,The Beatles,1969.0,13,15.000000
...,...,...,...,...
903,Yes,1972.0,1,35.000000
904,ZZ Top,1976.0,1,1.000000
905,ZZ Top,1980.0,1,34.000000
906,Zombies,1968.0,1,14.000000


In [23]:
observations_generator = pds.read_sql(query,
                            con,
                            coerce_float=True, # Doesn't efefct this dataset, because floats were correctly parsed
                            parse_dates=['Release_Year'], # Parse `Release_Year` as a date
                            chunksize=5 # Allows for streaming results as a series of shorter tables
                           )

for index, observations in enumerate(observations_generator):
    if index < 5:
        print(f'Observations index: {index}'.format(index))
        display(observations)

Observations index: 0


,Artist,Release_Year,num_of_songs,avg_play_time
0,The Beatles,1970-01-01 00:32:47,23,6.565217
1,Led Zeppelin,1970-01-01 00:32:49,18,21.000000
2,The Beatles,1970-01-01 00:32:45,15,3.800000
3,The Beatles,1970-01-01 00:32:48,13,13.000000
4,The Beatles,1970-01-01 00:32:49,13,15.000000


Observations index: 1


,Artist,Release_Year,num_of_songs,avg_play_time
0,Led Zeppelin,1970-01-01 00:32:50,12,13.166667
1,Led Zeppelin,1970-01-01 00:32:55,12,14.166667
2,Pink Floyd,1970-01-01 00:32:59,11,41.454545
3,Pink Floyd,1970-01-01 00:32:53,10,29.100000
4,The Doors,1970-01-01 00:32:47,10,28.900000


Observations index: 2


,Artist,Release_Year,num_of_songs,avg_play_time
0,Fleetwood Mac,1970-01-01 00:32:57,9,35.666667
1,Jimi Hendrix,1970-01-01 00:32:47,9,24.888889
2,The Beatles,1970-01-01 00:32:43,9,2.444444
3,The Beatles,1970-01-01 00:32:44,9,3.111111
4,Elton John,1970-01-01 00:32:53,8,18.500000


Observations index: 3


,Artist,Release_Year,num_of_songs,avg_play_time
0,Led Zeppelin,1970-01-01 00:32:51,8,47.750000
1,Led Zeppelin,1970-01-01 00:32:53,8,34.125000
2,Boston,1970-01-01 00:32:56,7,69.285714
3,Rolling Stones,1970-01-01 00:32:49,7,36.142857
4,Van Halen,1970-01-01 00:32:58,7,51.142857


Observations index: 4


,Artist,Release_Year,num_of_songs,avg_play_time
0,Bruce Springsteen,1970-01-01 00:32:55,6,7.666667
1,Bruce Springsteen,1970-01-01 00:33:04,6,11.500000
2,Creedence Clearwater Revival,1970-01-01 00:32:49,6,23.833333
3,Creedence Clearwater Revival,1970-01-01 00:32:50,6,18.833333
4,Def Leppard,1970-01-01 00:33:07,6,32.000000
